In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, TableColumn, DataTable
from bokeh.io import output_notebook
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

output_notebook()

import tkinter as tk
from tkinter import ttk
import hvplot.pandas
import panel as pn
pn.extension()
import holoviews as hv
hv.extension('bokeh')

# Statistics libraries
import statsmodels.api as sm
from ipywidgets import VBox

from scipy import stats
from scipy.stats import norm, shapiro, mannwhitneyu
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Output  # ✅ Corrected Import

from IPython.display import display




In [ ]:
# loading data using data_source module(individual module for loading data)
import data_source
data_sources = data_source.get_data() # dictionary
df_DSPH = pd.read_csv(data_sources['data_composite']) 


In [ ]:
df_DSPH.info()

In [ ]:
def miss_values(df):
    """
    Function to calculate the percentage of missing values (NaN) in each column 
    and the percentage of missing values in the entire DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame for which missing values are to be calculated.

    Returns:
    tuple: A tuple containing:
        - dic_miss_value (dict): A dictionary where keys are column names and values are the percentage of missing values in that column.
        - num_miss_df (float): The percentage of missing values in the entire DataFrame.
    """
    
    # Initialize an empty dictionary to store the missing values percentage for each column
    dic_miss_value = {}

    # Loop through each column in the DataFrame to calculate the percentage of missing values
    for col in df.columns:
        # Count the number of missing values (NaN) in the current column
        num_miss = df[col].isna().sum()

        # Calculate the percentage of missing values in the current column
        # Avoid division by zero if the DataFrame is empty
        percentage_miss = num_miss / len(df) if len(df) > 0 else 0

        # Store the percentage of missing values for the current column in the dictionary
        dic_miss_value[col] = percentage_miss

    # Calculate the percentage of missing values in the entire DataFrame
    # Avoid division by zero if the DataFrame is empty
    num_miss_df = df.isna().sum().sum() / df.size if df.size > 0 else 0

    # Return the dictionary of missing values percentages for each column and the overall missing value percentage
    return dic_miss_value, num_miss_df



In [ ]:
# Example usage:
dic_miss_value, num_miss_df = miss_values(df_DSPH)  # Missing values in the DataFrame
print(f'Missing values in each column: {dic_miss_value}')
print(f'Missing values in the whole DataFrame: {num_miss_df:.2%}')

In [ ]:
df_DSPH['Date1'] = pd.to_datetime(df_DSPH['Date1'], format="%Y-%m-%d")


In [ ]:
df_DSPH.tail()

From inspection, it could be observed that:
- the column Date1, Date2, Date3 & Date4 are all objects instead of dates
- Date1, Date2, Date3 & Date4 all contain the same values and therefore it is not necessary to maintain them 
- intake of dosage stopped for the 1st and 2nd people immediately the period for the research was over. 3rd and 4th people, however, continued with the intake of the dosage for a period of time after the end of the research. Therefore, 3rd and 4th have less missing values as compared to the first 2 people

# Data Cleaning

In [ ]:
def fill_clear(df, missing_dict):
    """
    Function to fill missing (NaN) values in specific columns of a DataFrame with the mean of those columns.
    It only fills numeric columns, and it creates a copy of the original DataFrame to avoid direct modification.

    Parameters:
    df (pandas.DataFrame): The input DataFrame where missing values need to be filled.
    missing_dict (dict): A dictionary where keys are column names and values are the strategy for filling missing values.
                         In this case, it expects the strategy to be 'mean' for each numeric column.

    Returns:
    pandas.DataFrame: A new DataFrame with missing values filled in the specified columns.
    """
    
    # Create a copy of the DataFrame to avoid modifying the original DataFrame directly
    df_copy = df.copy()

    # Loop through each key (column name) in the missing_dict
    for key in missing_dict.keys():
        # Check if the column exists in the DataFrame
        if key in df_copy.columns:
            # If the column is numeric, fill missing values with the mean of that column
            if pd.api.types.is_numeric_dtype(df_copy[key]):
                mean_value = df_copy[key].mean()  # Calculate the mean of the numeric column
                df_copy[key] = df_copy[key].fillna(mean_value)  # Fill missing values with the calculated mean
            else:
                # Print a message if the column is non-numeric and is skipped
                print(f"Skipping column '{key}' as it is non-numeric.")
        else:
            # Print a message if the column does not exist in the DataFrame
            print(f"Column '{key}' not found in DataFrame.")
    
    # Return the modified DataFrame with missing values filled
    return df_copy


In [ ]:
df_filled = fill_clear(df_DSPH,dic_miss_value)
print(df_filled)

In [ ]:
def remove_outliers_iqr(df, columns):
    """
    Function to remove outliers in specified columns of a DataFrame using the Interquartile Range (IQR) method.
    This method removes values that are outside the range defined by 1.5 times the IQR below the first quartile
    or above the third quartile.

    Parameters:
    df (pandas.DataFrame): The input DataFrame where outliers need to be removed.
    columns (list): A list of column names for which outliers will be removed using the IQR method.

    Returns:
    pandas.DataFrame: A new DataFrame with outliers removed from the specified columns.
    """
    
    # Create a copy of the DataFrame to avoid modifying the original DataFrame directly
    df_clean = df.copy()

    # Loop through each column in the specified columns list
    for col in columns:
        # Check if the column exists and is numeric
        if col in df_clean.columns and pd.api.types.is_numeric_dtype(df_clean[col]):
            # Calculate the first quartile (Q1) and third quartile (Q3)
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            
            # Calculate the Interquartile Range (IQR)
            IQR = Q3 - Q1
            
            # Determine the lower and upper bounds for outliers
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Filter the DataFrame to remove rows with values outside the bounds
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
        
        else:
            # Print a message if the column is not numeric or doesn't exist
            print(f"Skipping column '{col}' (not numeric or not found).")
    
    # Return the DataFrame with outliers removed
    return df_clean


In [ ]:
df_filled.info()

In [ ]:
# Clean the "Fat1","Fat2","Fat3","Fat4" columns
columns_to_clean = ['Fat1','Fat2','Fat3','Fat4','Muscle4']
df_filled[columns_to_clean]=df_filled[columns_to_clean].replace({r'%':'',r',':'.'},regex=True).apply(pd.to_numeric,errors='coerce')

df_filled = df_filled.astype({'Fat1': 'float', 'Fat2': 'float', 'Fat3': 'float','Fat4':'float'})


In [ ]:
df_filled.info()

In [ ]:
# colums to be investigated
used_columns = ['Fat1','Fat2','Fat3','Fat4',
                'Muscle1','Muscle2','Muscle3','Muscle4']

remove_outliers_iqr(df_filled, used_columns)

In [ ]:
df_filled.columns

In [ ]:
df_DSPH.columns = df_DSPH.columns.str.strip()
df_filled.columns = df_filled.columns.str.strip()

In [ ]:
df_filled.columns

In [ ]:
columns = ['Fat1', 'Fat2', 'Fat3', 'Fat4']
roman_labels = ['Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ']

plt.figure(figsize=(12, 8))

for i, (col, roman) in enumerate(zip(columns, roman_labels), 1):
    plt.subplot(2, 2, i) 
    stats.probplot(df_filled[col].dropna(), dist="norm", plot=plt) 
    plt.title(f'QQ Plot for {roman}') 

plt.tight_layout()
plt.show()

In [ ]:

# Define dosage periods
dosage_periods = {
    "First Half Dosage": (0, 10),
    "First Full Dosage": (11, 21),
    "Second Half Dosage": (21, 39),
    "Second Full Dosage": (39, len(df_filled))
}

# Function to compute mean values
def compute_means(df, column):
    return {dosage: df[column].iloc[start:end].mean() for dosage, (start, end) in dosage_periods.items()}

# Compute means for all persons
persons = ["Person I", "Person II", "Person III", "Person IV"]
fat_columns = ["Fat1", "Fat2", "Fat3", "Fat4"]
colors = ["blue", "red", "green", "purple"]

# Prepare Plotly figure
fig = go.Figure()

# Add traces for each person
for i, (person, fat_column, color) in enumerate(zip(persons, fat_columns, colors)):
    means_values = compute_means(df_filled, fat_column)
    
    fig.add_trace(go.Bar(
        x=list(means_values.keys()),  # Use category names instead of numbers
        y=list(means_values.values()),
        text=[f"{v:.2f}%" for v in means_values.values()],
        textposition="auto",
        name=person,
        marker=dict(color=color),
        visible=True if i == 0 else False  # Only Person I is visible initially
    ))

# Define visibility settings for dropdown menu
visibility_settings = [
    [True if i == j else False for j in range(len(persons))] for i in range(len(persons))
]

# Add dropdown menu for selecting persons
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {'label': person, 'method': 'update', 'args': [
                {'visible': visibility}, {'title': f'Comparison of Dosage Effects on % Fat Over Time ({person})'}
            ]}
            for person, visibility in zip(persons, visibility_settings)
        ],
        'direction': 'down',
        'showactive': True,
    }],
    title="Comparison of Dosage Effects on % Fat Over Time (Person I)",
    xaxis_title="Dosage Periods",
    yaxis_title="Mean % Fat",
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()


In [ ]:
# Define new columns
fat_percentage = ['Fat1', 'Fat2', 'Fat3', 'Fat4']
roman_labels = ['Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ']

# Convert to numeric to avoid errors
for col in  fat_percentage:
    df_filled[col] = pd.to_numeric(df_filled[col], errors='coerce')

# Plot Heart-rate
plt.figure(figsize=(12, 8))
for i, (col, roman) in enumerate(zip(fat_percentage, roman_labels), 1):
    plt.subplot(2, 2, i)
    sns.histplot(df_filled[col], kde=True, bins=15, color='orange')
    plt.title(f'Distribution of Percentage Fat - {roman}')
    plt.xlabel('% Fat')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
df_filled.columns

# PERCENTAGE MUSCLE

In [ ]:
# Clean the "Muscle1","Muscle2","Muscle3","Muscle4" columns
columns_to_clean = ['Muscle1','Muscle2','Muscle3','Muscle4']
df_filled[columns_to_clean]=df_filled[columns_to_clean].replace({r'%':'',r',':'.'},regex=True).apply(pd.to_numeric,errors='coerce')

df_filled = df_filled.astype({'Muscle1': 'float', 'Muscle2': 'float', 'Muscle3': 'float','Muscle4':'float'})


In [ ]:
# colums to be investigated
used_columns = ['Muscle1','Muscle2','Muscle3','Muscle4']

remove_outliers_iqr(df_filled, used_columns)

In [ ]:
df_DSPH.columns = df_DSPH.columns.str.strip()
df_filled.columns = df_filled.columns.str.strip()

In [ ]:
columns = ['Muscle1', 'Muscle2', 'Muscle3', 'Muscle4']
roman_labels = ['Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ']

plt.figure(figsize=(12, 8))

for i, (col, roman) in enumerate(zip(columns, roman_labels), 1):
    plt.subplot(2, 2, i) 
    stats.probplot(df_filled[col].dropna(), dist="norm", plot=plt) 
    plt.title(f'QQ Plot for {roman}') 

plt.tight_layout()
plt.show()

In [ ]:

# Define dosage periods
dosage_periods = {
    "First Half Dosage": (0, 10),
    "First Full Dosage": (11, 21),
    "Second Half Dosage": (21, 39),
    "Second Full Dosage": (39, len(df_filled))
}

# Function to compute mean values
def compute_means(df, column):
    return {dosage: df[column].iloc[start:end].mean() for dosage, (start, end) in dosage_periods.items()}

# Compute means for all persons
persons = ["Person I", "Person II", "Person III", "Person IV"]
muscle_columns = ["Muscle1", "Muscle2", "Muscle3", "Muscle4"]
colors = ["blue", "red", "green", "purple"]

# Prepare Plotly figure
fig = go.Figure()

# Add traces for each person
for i, (person, muscle_column, color) in enumerate(zip(persons, muscle_columns, colors)):
    means_values = compute_means(df_filled, muscle_column)
    
    fig.add_trace(go.Bar(
        x=list(means_values.keys()),  # Use category names instead of numbers
        y=list(means_values.values()),
        text=[f"{v:.2f}%" for v in means_values.values()],
        textposition="auto",
        name=person,
        marker=dict(color=color),
        visible=True if i == 0 else False  # Only Person I is visible initially
    ))

# Define visibility settings for dropdown menu
visibility_settings = [
    [True if i == j else False for j in range(len(persons))] for i in range(len(persons))
]

# Add dropdown menu for selecting persons
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {'label': person, 'method': 'update', 'args': [
                {'visible': visibility}, {'title': f'Comparison of Dosage Effects on % Muscle Over Time ({person})'}
            ]}
            for person, visibility in zip(persons, visibility_settings)
        ],
        'direction': 'down',
        'showactive': True,
    }],
    title="Comparison of Dosage Effects on % Muscle Over Time (Person I)",
    xaxis_title="Dosage Periods",
    yaxis_title="Mean % Muscle",
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()


In [ ]:
# Define new columns
muscle_percentage = ['Muscle1', 'Muscle2', 'Muscle3', 'Muscle4']
roman_labels = ['Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ']

# Convert to numeric to avoid errors
for col in  muscle_percentage:
    df_filled[col] = pd.to_numeric(df_filled[col], errors='coerce')

# Plot Heart-rate
plt.figure(figsize=(12, 8))
for i, (col, roman) in enumerate(zip(muscle_percentage, roman_labels), 1):
    plt.subplot(2, 2, i)
    sns.histplot(df_filled[col], kde=True, bins=15, color='orange')
    plt.title(f'Distribution of Percentage Fat - {roman}')
    plt.xlabel('% Fat')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()
